In [1]:
import cfbd
from cfbd.rest import ApiException
from pprint import pprint
import pandas as pd
import sqlite3
import DataFunctions
import numpy as np


configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = '3WCU5V2X05Rvh60ZxUG8FarJN4s2D1lcd2c2r6Kz/qL1Y3tVBJtWsuNATnzHRV2h'
configuration.api_key_prefix['Authorization'] = 'Bearer'

## Data Preprocessing

Games DF

In [2]:
api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))
year=2022
division="fbs"

# no longer need, right?
# try:
#     api_response = api_instance.get_games(year=year,division=division)
#     print(len(api_response))
# except:
#     print(ApiException)
    

In [3]:
gamelist = DataFunctions.get_fbs_games(api_instance=api_instance,year=2022)
len(gamelist)

735

In [4]:
games_df = DataFunctions.df_from_games(gamelist=gamelist)
games_df.head()

,id,season,neutral_site,home_id,home_team,home_conference,home_points,away_id,away_team,away_conference,away_points,game_spread,game_totalpts
0,401405059,2022,True,77,Northwestern,Big Ten,31,158,Nebraska,Big Ten,28,3,59
1,401405058,2022,False,356,Illinois,Big Ten,38,2751,Wyoming,Mountain West,6,32,44
2,401404146,2022,False,328,Utah State,Mountain West,31,41,Connecticut,FBS Independents,20,11,51
3,401426530,2022,False,2226,Florida Atlantic,Conference USA,43,2429,Charlotte,Conference USA,13,30,56
4,401426531,2022,False,2638,UTEP,Conference USA,13,249,North Texas,Conference USA,31,-18,44


In [5]:
conn = sqlite3.connect("CollegeFootball.db")
games_df.to_sql("games",conn,if_exists="replace",index=False)
conn.close()


In [6]:
# we can see with this code that we can extract DF from SQL db

# conn = sqlite3.connect("CollegeFootball.db")
# trialdf = pd.read_sql_query("SELECT * FROM games",conn)
# conn.close()
# print(trialdf.shape)
# trialdf.head()


Betting lines DF

In [7]:
api_instance1 = cfbd.BettingApi(cfbd.ApiClient(configuration))

In [8]:
conferences=[]
for game in gamelist:
    conferences.append(game.away_conference)
conferences=set(conferences)

conferences

{'ACC',
 'American Athletic',
 'Big 12',
 'Big Ten',
 'Conference USA',
 'FBS Independents',
 'Mid-American',
 'Mountain West',
 'Pac-12',
 'SEC',
 'Sun Belt'}

In [9]:
betting_list=DataFunctions.get_fbs_betting(api_instance=api_instance1,year=2022,conferences=conferences)

In [10]:
betting_df=DataFunctions.df_betting_lines(betting_list)
betting_df

#warning seems to be since betting line -0.0 for one of the games but still stores value as 0 so should be fine

c:\Users\rodri\anaconda3\envs\PIC16B\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\rodri\anaconda3\envs\PIC16B\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,id,av_spread,av_total
0,401411101,-24.375,51.375
1,401404079,7.125,65.625
2,401403893,1.500,51.000
3,401404080,2.500,45.000
4,401404081,13.625,53.000
...,...,...,...
733,401403935,12.500,50.500
734,401404029,21.125,61.125
735,401416641,-2.750,55.750
736,401404036,11.500,47.000


In [11]:
conn=sqlite3.connect("CollegeFootball.db")
betting_df.to_sql("betting_lines",conn,if_exists="replace",index=False)
conn.close()

In [12]:
#sample merge dataframe

cmd=\
"""
SELECT G.away_team,G.away_points,G.home_points,G.home_team,B.av_spread,B.av_total
FROM games G
INNER JOIN betting_lines B ON G.id=B.id
"""
conn=sqlite3.connect("CollegeFootball.db")
test_merge=pd.read_sql_query(cmd,conn)
conn.close() # do not forget this after query
test_merge

,away_team,away_points,home_points,home_team,av_spread,av_total
0,Nebraska,28,31,Northwestern,-12.375,52.250
1,Wyoming,6,38,Illinois,14.000,42.750
2,Connecticut,20,31,Utah State,23.750,59.250
3,Charlotte,13,43,Florida Atlantic,7.000,60.000
4,North Texas,31,13,UTEP,-1.375,52.875
...,...,...,...,...,...,...
730,Fresno State,28,16,Boise State,3.000,54.750
731,LSU,30,50,Georgia,17.125,51.750
732,Clemson,39,10,North Carolina,-7.250,64.000
733,Purdue,22,43,Michigan,15.625,53.750


Team stats per season DF

In [13]:
api_instance2 = cfbd.StatsApi(cfbd.ApiClient(configuration))

In [14]:
teamstats = api_instance2.get_advanced_team_season_stats(year=year)

In [15]:
stats_df = DataFunctions.df_team_advstats(teamstats=teamstats)

In [16]:
stats_df.head()

,team,season,conference,Offensive plays,Offensive drives,Offensive ppa,Offensive total_ppa,Offensive success_rate,Offensive explosiveness,Offensive power_success,...,Defensive passing_downs explosiveness,Defensive rushing_plays rate,Defensive rushing_plays ppa,Defensive rushing_plays total_ppa,Defensive rushing_plays success_rate,Defensive rushing_plays explosiveness,Defensive passing_plays rate,Defensive passing_plays ppa,Defensive passing_plays success_rate,Defensive passing_plays explosiveness
0,Air Force,2022,Mountain West,772,120,0.297878,229.961644,0.480570,1.132888,0.825000,...,1.977848,0.516129,0.097045,29.501558,0.394737,0.969436,0.466893,0.255682,0.425455,1.410758
1,Akron,2022,Mid-American,615,109,0.113974,70.093797,0.426016,1.185226,0.735294,...,2.176663,0.555944,0.156959,49.913104,0.446541,1.005307,0.437063,0.449689,0.480000,1.656349
2,Alabama,2022,SEC,895,160,0.305324,273.264690,0.471508,1.375730,0.722222,...,1.917980,0.472036,0.078791,33.249994,0.407583,0.952710,0.525727,0.012309,0.348936,1.379126
3,Appalachian State,2022,Sun Belt,864,147,0.259159,223.913126,0.456019,1.295246,0.705882,...,2.260158,0.493011,0.136812,53.082881,0.402062,1.053082,0.499365,0.313190,0.430025,1.543410
4,Arizona,2022,Pac-12,788,141,0.271661,214.068935,0.472081,1.379341,0.673469,...,2.029388,0.507335,0.338295,140.392468,0.559036,0.998885,0.487775,0.389756,0.501253,1.398465


In [24]:
stats_df.columns

Index(['team', 'season', 'conference', 'Offensive plays', 'Offensive drives',
       'Offensive ppa', 'Offensive total_ppa', 'Offensive success_rate',
       'Offensive explosiveness', 'Offensive power_success',
       'Offensive stuff_rate', 'Offensive line_yards',
       'Offensive line_yards_total', 'Offensive second_level_yards',
       'Offensive second_level_yards_total', 'Offensive open_field_yards',
       'Offensive open_field_yards_total', 'Offensive total_opportunies',
       'Offensive points_per_opportunity', 'Defensive plays',
       'Defensive drives', 'Defensive ppa', 'Defensive total_ppa',
       'Defensive success_rate', 'Defensive explosiveness',
       'Defensive power_success', 'Defensive stuff_rate',
       'Defensive line_yards', 'Defensive line_yards_total',
       'Defensive second_level_yards', 'Defensive second_level_yards_total',
       'Defensive open_field_yards', 'Defensive open_field_yards_total',
       'Defensive total_opportunies', 'Defensive points_p

In [20]:
conn = sqlite3.connect("CollegeFootball.db")
stats_df.to_sql("team_stats",conn,if_exists="replace",index=False)
conn.close()

In [22]:
# conn = sqlite3.connect("CollegeFootball.db")
# trialdf = pd.read_sql_query("SELECT * FROM team_stats",conn)
# conn.close()
# print(trialdf.shape)
# trialdf.head()

Merging tables to set up for NN

In [ ]:
# things we are missing for neural network: Strength of Schedule
# either grab it from somewhere, create it using our own data or use the 
# data that takes it into account.